<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/Feature%20Engineering/featureEngineeringMMElo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [1]:
#Import Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import mode
import pickle
import datetime 

## Mounting Drive

In [2]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set up the path

In [3]:
%cd /content/drive/MyDrive/data

/content/drive/MyDrive/data


### View Current Items in the folder

In [4]:
!ls

 augmented_test.csv
 augmented_test_try1.csv
 augmented_test_try1_model_imputation.csv
 augmented_train.csv
 augmented_train_try1.csv
 augmented_train_try1_model_imputation.csv
 clf_cat2.sav
 clf_cat3.sav
 clf_name.sav
'Data Dictionary.xlsx'
 Data_Dictionary.xlsx
 file_name.csv
 hist_authorized_flag_enc.npy
 hist_category_1_enc.npy
 hist_category_3_enc.npy
 historical_transactions.csv
 historical_transactions_df.pkl
 hist_trans_preprocessed.csv
 knn_sal_12.sav
 knn_sal_3.sav
 knn_sal_6.sav
 LR_cat_2.sav
 mer_authorized_flag_enc.npy
 mer_category_1_enc.npy
 mer_category_3_enc.npy
 merchant_category_1_enc.npy
 merchant_category_4_enc.npy
 merchant_most_recent_purchases_range_enc.npy
 merchant_most_recent_sales_range_enc.npy
 merchants.csv
 merchants_preprocessed.csv
 merged_transactions_with_merchants.csv
 new_merchant_authorized_flag_enc.npy
 new_merchant_category_1_enc.npy
 new_merchant_df.pkl
 newMerchants_trans_preprocessed.csv
 new_merchant_transactions.csv
 sample_submission.csv
 t

## Helper Functions

### Timer

In [6]:
def reduce_mem_usage(df, verbose=True):
  """ Helps to reduce memory usage """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [7]:
def dateTimeFeatures(df):
  """

    To retuurn a dataframe with new date time features 
    ---------------------------------------------------------
    Args:
      df:- The dataframe on which we want to do 
    ---------------------------------------------------------
    Returns:
    A dataframe with date time features 
    ---------------------------------------------------------
    To DO:
      1. Add holidays 

  """

  # basic date time features
  df['month'] = df.purchase_date.dt.month
  df['day'] = df.purchase_date.dt.day
  df['hour'] = df.purchase_date.dt.hour
  df['weekofyear'] = df.purchase_date.dt.weekofyear
  df['weekend'] = (df.purchase_date.dt.weekday >= 5).astype(int)
  df['weekday'] = df.purchase_date.dt.weekday
  #difference in months from today
  df['monthDiff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days) // 30

  return df

In [8]:
def aggregations(df):
  """
    Find the group aggregations 

  """
  
  df = reduce_mem_usage(df)

  #define a function for mode 
  def mode(series):
      """Most common element in a series"""
      tdmode = series.mode()
      if len(tdmode) == 0:
          return np.nan
      else:
          return tdmode[0]

  #define a function for peroid
  def period(series):
      """Period of a series (max-min)"""
      return series.max() - series.min()

  #price is the purchace amount by the number of installments
  df['price'] = df['purchase_amount']/df['installments']

  # find the date time features
  df = dateTimeFeatures(df)

  aggs = {}

  #getting the nunique for the following ID based features  
  cols = ['city_id', 'state_id', 'merchant_category_id', 'subsector_id', 'merchant_id']
  colsMer = ['mer_city_id', 'mer_state_id', 'mer_merchant_category_id', 'mer_subsector_id','mer_merchant_group_id']
  
  for col in cols:
    aggs[col] = ['nunique']
  for col in colsMer:
    aggs[col] = ['nunique']


  #2. categorical  variables 
  aggs['authorized_flag'] = ['sum']
  aggs['category_1'] = ['sum', 'mean', mode]
  aggs['category_2'] = ['sum', 'mean', mode]
  aggs['category_3'] = ['sum', 'mean', mode]
  aggs['mer_category_1'] = ['sum', 'mean', mode]
  aggs['mer_category_2'] = ['sum', 'mean', mode]
  aggs['mer_category_4'] = ['sum', 'mean', mode]
  aggs['mer_most_recent_purchases_range'] = ['sum', 'mean', mode]
  aggs['mer_most_recent_sales_range'] = ['sum', 'mean', mode]

  #3. Continues variables
  aggs['price'] = [ 'max', 'min']
  aggs['mer_avg_sales_lag3'] = ['mean', 'max', 'min', 'var']
  aggs['mer_avg_purchases_lag3'] = ['mean', 'max', 'min', 'var']
  aggs['mer_active_months_lag3'] = ['mean', 'max', 'min', 'var']
  aggs['mer_avg_sales_lag6'] = ['mean', 'max', 'min', 'var']
  aggs['mer_avg_purchases_lag6'] = ['mean', 'max', 'min', 'var']
  aggs['mer_active_months_lag6'] = ['mean', 'max', 'min', 'var']
  aggs['mer_avg_sales_lag12'] = ['mean', 'max', 'min', 'var']
  aggs['mer_avg_purchases_lag12'] = ['mean', 'max', 'min', 'var']
  aggs['mer_active_months_lag12'] = ['mean', 'max', 'min', 'var']
  aggs['mer_numerical_1'] = ['mean', 'max', 'min', 'var']
  aggs['mer_numerical_2'] = ['mean', 'max', 'min', 'var']

  #4. Date Time 
  aggs['month'] = ['sum','mean', 'nunique','min','max',period]
  aggs['day'] = ['sum','mean', 'nunique','min','max',period]
  aggs['hour'] = ['sum','mean', 'nunique','min','max',period]
  aggs['weekofyear'] = ['sum','mean', 'nunique','min','max',period]
  aggs['weekend'] = ['sum','mean', 'nunique','max', 'min',period]
  aggs['weekday'] = ['sum','mean', 'nunique','max', 'min',period]
  aggs['month_lag'] = ['mean', 'min', 'max', 'var', 'skew',period]

  df = df.reset_index().groupby('card_id').agg(aggs)
  print('[INFO]  DONE ...')
  
  return df

In [9]:
def nullValues(df):
  """ Find the columns with null values """
  columns_with_na = [var for var in df.columns if df[var].isnull().mean()  > 0]
  #lets find out the percentage of observations missing per variable

  #calculate the percentage of missing 
  data_na = df[columns_with_na].isnull().mean()

  #transform the array to dataframe 
  data_na = pd.DataFrame(data_na.reset_index())

  #add names to the dataframe 
  data_na.columns = ['col','percentage_na']

  #oreder the dataframe acc to percentage 
  data_na.sort_values(by = 'percentage_na',ascending = False, inplace = True)

  #show
  data_na
  return data_na

## Merge and appply

In [10]:
histDf = pd.read_csv("hist_trans_preprocessed.csv",parse_dates=['purchase_date']) 

newdf = pd.read_csv("newMerchants_trans_preprocessed.csv",parse_dates=['purchase_date']) 

merchants = pd.read_csv("merchants_preprocessed.csv") 
merchants.columns = 'mer_' + merchants.columns.values
merchants.rename(columns = {'mer_merchant_id':'merchant_id'}, inplace = True) 

# Merge transactions with merchants data
histDf = pd.merge(histDf, merchants, on='merchant_id')
newdf = pd.merge(newdf, merchants, on='merchant_id')

# Clean up
del merchants
gc.collect()

histDf = aggregations(histDf)
histDf.columns = ["_".join(col) for col in histDf.columns.ravel()]
newdf = aggregations(newdf)
newdf.columns = ["_".join(col) for col in newdf.columns.ravel()]

Memory usage after optimization is: 2620.48 MB
Decreased by 68.4%
[INFO]  DONE ...
Memory usage after optimization is: 167.10 MB
Decreased by 69.1%
[INFO]  DONE ...


In [11]:
nullValues(histDf)

,col,percentage_na
0,month_lag_skew,0.00019


In [12]:
nullValues(newdf)

,col,percentage_na
12,month_lag_skew,0.273600
0,mer_avg_sales_lag3_var,0.145176
1,mer_avg_purchases_lag3_var,0.145176
2,mer_active_months_lag3_var,0.145176
3,mer_avg_sales_lag6_var,0.145176
4,mer_avg_purchases_lag6_var,0.145176
5,mer_active_months_lag6_var,0.145176
6,mer_avg_sales_lag12_var,0.145176
7,mer_avg_purchases_lag12_var,0.145176
8,mer_active_months_lag12_var,0.145176


In [13]:
histDf

,city_id_nunique,state_id_nunique,merchant_category_id_nunique,subsector_id_nunique,merchant_id_nunique,mer_city_id_nunique,mer_state_id_nunique,mer_merchant_category_id_nunique,mer_subsector_id_nunique,mer_merchant_group_id_nunique,authorized_flag_sum,category_1_sum,category_1_mean,category_1_mode,category_2_sum,category_2_mean,category_2_mode,category_3_sum,category_3_mean,category_3_mode,mer_category_1_sum,mer_category_1_mean,mer_category_1_mode,mer_category_2_sum,mer_category_2_mean,mer_category_2_mode,mer_category_4_sum,mer_category_4_mean,mer_category_4_mode,mer_most_recent_purchases_range_sum,mer_most_recent_purchases_range_mean,mer_most_recent_purchases_range_mode,mer_most_recent_sales_range_sum,mer_most_recent_sales_range_mean,mer_most_recent_sales_range_mode,price_max,price_min,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_max,mer_avg_sales_lag3_min,...,month_nunique,month_min,month_max,month_period,day_sum,day_mean,day_nunique,day_min,day_max,day_period,hour_sum,hour_mean,hour_nunique,hour_min,hour_max,hour_period,weekofyear_sum,weekofyear_mean,weekofyear_nunique,weekofyear_min,weekofyear_max,weekofyear_period,weekend_sum,weekend_mean,weekend_nunique,weekend_max,weekend_min,weekend_period,weekday_sum,weekday_mean,weekday_nunique,weekday_max,weekday_min,weekday_period,month_lag_mean,month_lag_min,month_lag_max,month_lag_var,month_lag_skew,month_lag_period
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,4,3,18,13,29,2,2,18,12,14,114.0,28.0,0.187919,0,393.0,2.636719,3.0,173.0,1.161074,1,121.0,0.812081,1,391.0,2.625000,3.0,100.0,0.671141,1,281.0,1.885906,3,268.0,1.798658,3,0.502356,-0.728876,0.965906,1.25,0.62,...,12,1,12,11,1974,13.248322,28,1,31,30,2148,14.416107,18,0,22,22,3807,25.550336,39,1,52,51,25,0.167785,2,1,0,1,334,2.241611,7,6,0,6,-5.852349,-12,0,11.923998,0.135031,12
C_ID_0001238066,18,6,29,17,65,16,5,27,17,50,120.0,2.0,0.016260,0,203.0,1.650391,1.0,152.0,1.235772,1,118.0,0.959350,1,195.0,1.584961,1.0,103.0,0.837398,1,203.0,1.650407,2,225.0,1.829268,2,0.605012,-0.734887,1.184716,16.18,0.77,...,6,1,12,11,1985,16.138211,30,1,30,29,1813,14.739837,20,0,23,23,3686,29.967480,23,1,52,51,52,0.422764,2,1,0,1,431,3.504065,7,6,0,6,-1.813008,-5,0,1.661469,-0.239327,5
C_ID_0001506ef0,3,2,19,12,28,3,2,17,12,18,64.0,0.0,0.000000,0,196.0,2.882812,3.0,1.0,0.014706,0,64.0,0.941176,1,196.0,2.882812,3.0,1.0,0.014706,0,89.0,1.308824,0,90.0,1.323529,0,inf,-inf,1.207353,6.93,0.72,...,11,1,12,11,810,11.911765,25,1,31,30,856,12.588235,15,0,21,21,1886,27.735294,24,1,51,50,34,0.500000,2,1,0,1,246,3.617647,7,6,0,6,-4.750000,-13,0,17.652985,-0.771502,13
C_ID_0001793786,10,4,48,24,119,7,4,46,23,85,195.0,2.0,0.009009,0,328.0,1.477539,1.0,5.0,0.022523,0,190.0,0.855856,1,416.0,1.874023,2.0,65.0,0.292793,0,455.0,2.049550,2,443.0,1.995495,2,inf,-inf,1.379820,26.91,0.55,...,10,1,10,9,3592,16.180180,31,1,31,30,3480,15.675676,21,0,23,23,6031,27.166667,33,3,44,41,37,0.166667,2,1,0,1,591,2.662162,7,6,0,6,-3.324324,-9,0,5.269903,-0.402812,9
C_ID_000183fdda,9,7,36,21,73,3,4,35,20,57,142.0,4.0,0.026846,0,416.0,2.791016,3.0,182.0,1.221477,1,129.0,0.865772,1,407.0,2.732422,3.0,7.0,0.046980,0,308.0,2.067114,2,284.0,1.906040,1,0.936062,-0.737892,1.371477,10.67,0.61,...,7,1,12,11,2077,13.939597,30,1,31,30,2462,16.523490,19,0,23,23,4151,27.859060,27,1,52,51,35,0.234899,2,1,0,1,436,2.926174,7,6,0,6,-2.449664,-6,0,3.614003,-0.219218,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_ID_ffff1d9928,3,3,4,4,8,2,2,4,4,5,10.0,1.0,0.083333,0,39.0,3.250000,4.0,13.0,1.083333,1,9.0,0.750000,1,39.0,3.250000,4.0,9.0,0.750000,1,26.0,2.166667,3,30.0,2.500000,3,0.416325,-0.746893,1.500833,6.93,0.88,...,4,1,12,11,192,16.000000,7,5,31,26,145,12.

In [14]:
newdf

,city_id_nunique,state_id_nunique,merchant_category_id_nunique,subsector_id_nunique,merchant_id_nunique,mer_city_id_nunique,mer_state_id_nunique,mer_merchant_category_id_nunique,mer_subsector_id_nunique,mer_merchant_group_id_nunique,authorized_flag_sum,category_1_sum,category_1_mean,category_1_mode,category_2_sum,category_2_mean,category_2_mode,category_3_sum,category_3_mean,category_3_mode,mer_category_1_sum,mer_category_1_mean,mer_category_1_mode,mer_category_2_sum,mer_category_2_mean,mer_category_2_mode,mer_category_4_sum,mer_category_4_mean,mer_category_4_mode,mer_most_recent_purchases_range_sum,mer_most_recent_purchases_range_mean,mer_most_recent_purchases_range_mode,mer_most_recent_sales_range_sum,mer_most_recent_sales_range_mean,mer_most_recent_sales_range_mode,price_max,price_min,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_max,mer_avg_sales_lag3_min,...,month_nunique,month_min,month_max,month_period,day_sum,day_mean,day_nunique,day_min,day_max,day_period,hour_sum,hour_mean,hour_nunique,hour_min,hour_max,hour_period,weekofyear_sum,weekofyear_mean,weekofyear_nunique,weekofyear_min,weekofyear_max,weekofyear_period,weekend_sum,weekend_mean,weekend_nunique,weekend_max,weekend_min,weekend_period,weekday_sum,weekday_mean,weekday_nunique,weekday_max,weekday_min,weekday_period,month_lag_mean,month_lag_min,month_lag_max,month_lag_var,month_lag_skew,month_lag_period
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2,2,2,2,2,2,2,2,2,2,3,0,0.000000,0,5.0,1.666992,1.0,3,1.000000,1,1,0.333333,0,5.0,1.666992,1.0,0,0.000000,0,8.0,2.666667,4,7.0,2.333333,0,-0.656738,-0.671875,2.920000,6.93,0.90,...,1,4,4,0,21,7.000000,2,3,9,6,43,14.333333,2,11,16,5,44,14.666667,2,14,15,1,0,0.000000,1,0,0,0,1,0.333333,2,1,0,1,2.000000,2,2,0.000000,0.000000,0
C_ID_0001238066,8,4,14,9,25,7,4,14,9,19,26,2,0.076923,0,38.0,1.461914,1.0,29,1.115385,1,20,0.769231,1,38.0,1.461914,1.0,16,0.615385,1,46.0,1.769231,0,44.0,1.692308,0,0.151978,-0.740723,1.226923,6.93,0.81,...,2,3,4,1,480,18.461538,14,1,30,29,384,14.769231,15,0,23,23,335,12.884615,9,9,18,9,11,0.423077,2,1,0,1,104,4.000000,6,6,0,6,1.346154,1,2,0.235385,0.687052,1
C_ID_0001506ef0,1,1,1,1,1,1,1,1,1,1,1,0,0.000000,0,3.0,3.000000,3.0,0,0.000000,0,1,1.000000,1,3.0,3.000000,3.0,0,0.000000,0,2.0,2.000000,2,2.0,2.000000,2,-inf,-inf,1.040000,1.04,1.04,...,1,3,3,0,22,22.000000,1,22,22,0,9,9.000000,1,9,9,0,12,12.000000,1,12,12,0,0,0.000000,1,0,0,0,3,3.000000,1,3,3,0,1.000000,1,1,NaN,NaN,0
C_ID_0001793786,7,5,21,14,31,4,4,21,14,21,31,0,0.000000,0,53.0,1.709961,1.0,0,0.000000,0,27,0.870968,1,51.0,1.645508,1.0,9,0.290323,0,65.0,2.096774,3,60.0,1.935484,3,inf,-inf,1.205806,4.00,0.61,...,2,11,12,1,732,23.612903,13,10,31,21,354,11.419355,10,0,21,21,1500,48.387097,6,46,52,6,14,0.451613,2,1,0,1,111,3.580645,6,6,0,6,1.322581,1,2,0.225806,0.798227,1
C_ID_000183fdda,2,2,9,6,11,2,2,9,6,9,11,0,0.000000,0,33.0,3.000000,3.0,14,1.272727,1,11,1.000000,1,33.0,3.000000,3.0,0,0.000000,0,25.0,2.272727,3,28.0,2.545455,2,0.107666,-0.732422,1.059091,1.30,0.78,...,2,3,4,1,129,11.727273,9,2,30,28,170,15.454545,8,11,22,11,130,11.818182,7,9,18,9,2,0.181818,2,1,0,1,27,2.454545,6,5,0,5,1.272727,1,2,0.218182,1.189373,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_ID_ffff1d9928,2,2,4,4,4,2,2,4,4,4,4,1,0.250000,0,13.0,3.250000,4.0,5,1.250000,1,3,0.750000,1,13.0,3.250000,4.0,3,0.750000,1,7.0,1.750000,2,6.0,1.500000,2,-0.028564,-0.729004,1.010000,1.17,0.90,...,2,3,4,1,75,18.750000,3,16,20,4,56,14.000000,4,7,21,14,60,15.000000,2,12,16,4,0,0.000000,1,0,0,0,8,2.000000,2,4,0,4,1.750000,1,2,0.250000,-2.000000,1
C_ID_ffff579d3a,1,1,1,1,1,1,1,1,1,1,1,0,0.000000,0,1.0,1.000000,1.0,0,0.000000,0,1,1.000000,1,1.0,1.000000,1.0